In [1]:
from ipywidgets import *
from ipywebrtc import CameraStream, ImageRecorder
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
from ipywidgets import HTML
import subprocess, sys
import base64
import cv2
import io
import numpy as np
import PIL.Image
from PIL import *

In [2]:
gamma = 1
contrast_init = 0

In [3]:
####  Inversion  #####
def invert(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################
    iminvers = cv2.bitwise_not(imm)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(iminvers, fmt)
    Img.value=data_comp

In [4]:
#################################
def Canny_transfo(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################"

    # Setting parameter values
    t_lower = 50  # Lower Threshold
    t_upper = 150  # Upper threshold

    # Applying the Canny Edge filter
    edge = cv2.Canny(imm, t_lower, t_upper)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(edge, fmt)
    Img.value=data_comp

In [5]:
#################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########

In [6]:
#############################################
def adjust_brightness(img, brightness_factor):
    print("brightness_factor = ",brightness_factor)
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################

In [7]:
def Travail_img(gamma, contrast_init):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    imm=adjust_brightness(imm, gamma)
    #########################################"
    fmt = 'png'
    data_comp = compress_to_bytes(imm, fmt)
    Img.value=data_comp
    return gamma, contrast_init
    #####

In [8]:
def Contrast_w(_):
    global gamma, contrast_init
    contrast=contraste_widg.value
#     print("contraste lu = ",contrast)
#     print("contrast_init = ",contrast_init)
    if contrast > contrast_init:

        gamma=float((contrast-contrast_init)/10)+1
#         print("* Gamma = ",gamma)
        contrast_init = contrast
#         print("************** New contrast-init = ",contrast_init)
    else:
        gamma=1-float((contrast_init-contrast)/10)
#         print("* Gamma2 = ",gamma)
        contrast_init = contrast
#         print("************** New contrast-init2 = ",contrast_init)
    
    Travail_img(gamma, contrast_init)

In [9]:
######  Recup Image d'1 Vidéo  ########
def Recup_image(_):
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)#[..., :3]
    ####
#     mask_circle = np.zeros(im_array.shape[:2],np.uint8)
#     cv2.circle(mask_circle,(320,240), 50, 255, -1) # a.value, 255, -1)
#     img_rgb = cv2.bitwise_and(im_array, im_array, mask=mask_circle)    
    ####
    fmt = 'png'
    data_comp = compress_to_bytes(im_array, fmt)
    Img.value = data_comp
    Img_rescue.value = data_comp

    ####
#     data_comp2 = compress_to_bytes(im_array, fmt)
#     image_headline2.value = data_comp2
    yo.layout.visibility = 'hidden'
#     Mode_w.value="---"
#################################

In [10]:
#######################################
def upload_func(change):
    valeur=change['new']
#     global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode, Trait,Down
    if valeur=={}:
        rien=1
    else:
        im_val = uplo.value
        cles = list(im_val.keys())
#         print(cles)
        cle_one=cles[0]
        conta = im_val[cle_one]
        contenu_ex_taille = conta['content']
        nparr = np.frombuffer(contenu_ex_taille, np.uint8)
        img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  # cv2.CV_LOAD_IMAGE_COLOR) # cv2.IMREAD_COLOR in OpenCV 3.1   
        pil_im=cv2.resize(img_np, (640,480), interpolation = cv2.INTER_NEAREST)
        pil_im = cv2.cvtColor(pil_im, cv2.COLOR_BGR2RGB)
#         shap = pil_im.shape

        pil_array=np.array(pil_im)
        fmt = 'png'
        data_image_headline = compress_to_bytes(pil_array, fmt)
            
        Img.value=data_image_headline
        Img_rescue.value=data_image_headline 

#         Iris_ok=1
        Travail_img(gamma, contrast_init)
#############################################

In [11]:
##########
def arrete_tout(_):
    Widget.close_all()

In [12]:
###########
def reinitialise(_):
    Img.value = Img_rescue.value

In [13]:
###########
def Capture_w_fonc(_):
    yo.layout.display = "block"

<div style="text-align:center;  font-size: 3.2em; width:70%;">
<span style="color:brown"><b><u>APPLICATIONS EN LIGNE</u></b></span>
</div>



<div style="text-align:center; border:2px solid black; font-size: 1.8em; width:40%;">
<span style="color:blue"><b>Application Web de retouche d'images :</b></span>
</div>



In [14]:
file = open("Lycee_NSI.jpg", "rb")
image = file.read()
Img = widgets.Image(
    value=image,
    format='jpg',
    width=600,
    height=550,
)
Img_rescue = widgets.Image(
    value=image,
    format='jpg',
    width=600,
    height=550,
)

In [15]:
contraste_widg = widgets.IntSlider(min = -10, max = +10,  value = 0 , description='+ clair')
contraste_widg.observe(Contrast_w, names='value')

In [16]:
#### Stop  ####
Stop_w = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour fermer l'Appli",
    icon='times-circle' # (FontAwesome names without the `fa-` prefix)
)
Stop_w.observe(arrete_tout,names='value')
#### Rescue  ####
Rescue_w = widgets.ToggleButton(
    value=False,
    description='Réinitialise',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour revenir à l'image référence",
    icon='undo' # (FontAwesome names without the `fa-` prefix)
)
Rescue_w.observe(reinitialise,names='value')
#####################

In [17]:
####  Dessin  #####
Cany_w = widgets.ToggleButton(
    value=False,
    description='Dessin',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour obtenir le dessin de contour",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Cany_w.observe(Canny_transfo,names='value')
#####################

In [18]:
####  Invert  #####
Invert_w = widgets.ToggleButton(
    value=False,
    description='Inverse',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour inverser le noir et le blanc",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Invert_w.observe(invert,names='value')
#####################

In [19]:
###########################################
uplo = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    icon="upload"
)
uplo.layout.min_width = '49%'
uplo.layout.min_height = '42px'
uplo.layout.border = 'thick double #CCCCFF'
uplo.observe(upload_func, names='value')
###############################

In [20]:
##################
Capture_w = widgets.ToggleButton(
    value=False,
    description='Capture',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour réaliser une capture Webcam",
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)
Capture_w.observe(Capture_w_fonc,names='value')
########################################

In [21]:
# Contraste_Boite = HBox([ contraste_widg, clair_z,sombre_z], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
Contraste_Boite = HBox([ contraste_widg], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))

Cany_invert_Boite = HBox([Cany_w, Invert_w], layout=Layout(margin='0 0 0 0px'))
Controle_Boite = HBox([Stop_w, Rescue_w])
Img_src_Boite = HBox([uplo, Capture_w ])
Menu_Boite1 = VBox([Controle_Boite, Img_src_Boite, Contraste_Boite, Cany_invert_Boite])
Menus = HBox([Img, Menu_Boite1], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

In [22]:
camera = CameraStream(constraints={'facingMode': {"exact": "environment"},'audio': False,'video': { 'width': 640, 'height': 480}})
# camera.layout.width='20%'
image_recorder = ImageRecorder(stream=camera, tooltip='Cliquer pour tenter une Capture')

btn_stop = Button(description="Envois à l'Iris",tooltip='Cette capture est Ok ?',button_style='success', icon="eye")
btn_stop.on_click(Recup_image)

yo=HBox([camera,VBox([btn_stop,image_recorder])], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))

yo.layout.display = 'none'
display(yo)

<div style="text-align:center; border:2px solid black; font-size: 1.8em; width:55%;">
<span style="color:blue"><b>Application Web de Travail sur l'encodage :</b></span>
</div>

In [23]:
def Dec_Bin_Hex(_):
    Nombre = Nombre_utilisateur.value
    if Nombre != "":
        try:
            Nombre = int(Nombre)
            if Nombre >=0:
                Pre_Nb_bin = bin(Nombre)
                Nb_bin = str(Pre_Nb_bin[2:])
                Pre_Nb_hex = hex(Nombre)
                Nb_hex = str(Pre_Nb_hex[2:])
                phrase_bin = "L'encodage, en binaire, est : " + Nb_bin
                Encod_Dec_Bin_label.value = phrase_bin
                phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                Encod_Dec_Hex_label.value = phrase_hex
            else:
                phrase_bin = "L'encodage, en binaire, est : "
                Encod_Dec_Bin_label.value = phrase_bin 
                phrase_hex = "L'encodage, en hexadécimal, est : "
                Encod_Dec_Hex_label.value = phrase_hex
        except ValueError:
            phrase_bin = "ATTENTION, vous devez entrer un NOMBRE ENTIER POSITIF."
            Encod_Dec_Bin_label.value = phrase_bin
            phrase_hex = "ATTENTION, vous devez entrer un NOMBRE ENTIER POSITIF."
            Encod_Dec_Hex_label.value = phrase_hex
    else:
        phrase_bin = "L'encodage, en binaire, est : "
        Encod_Dec_Bin_label.value = phrase_bin 
        phrase_hex = "L'encodage, en hexadécimal, est : "
        Encod_Dec_Hex_label.value = phrase_hex

In [24]:
def Bin_Dec_Hex(_):
    Nombre = Nombre_utilisateur2.value
    if Nombre != "":
        try:
            Nombre = int(Nombre)
            Nombre_0b = "0b" + str(Nombre)
            if Nombre >=0:
                try:
                    Pre_Nb_dec = int(Nombre_0b,2)
                    Nb_dec = str(Pre_Nb_dec) 
                    Pre_Nb_hex = hex(Pre_Nb_dec)
                    Nb_hex = str(Pre_Nb_hex[2:])
                    phrase_dec = "L'encodage, en décimal, est : " + Nb_dec
                    Encod_Bin_Dec_label.value = phrase_dec
                    phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                    Encod_Bin_Hex_label.value = phrase_hex
                except ValueError :
                    phrase_dec = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
                    Encod_Bin_Dec_label.value = phrase_dec
                    phrase_hex = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
                    Encod_Bin_Hex_label.value = phrase_hex
            else:
                phrase_dec = "L'encodage, en décimal, est : "
                Encod_Bin_Dec_label.value = phrase_dec
                phrase_hex = "L'encodage, en hexadécimal, est : " + Nb_hex
                Encod_Bin_Hex_label.value = phrase_hex
        except ValueError:
            phrase_dec = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
            Encod_Bin_Dec_label.value = phrase_dec
            phrase_hex = "ATTENTION, vous devez entrer un NOMBRE BINAIRE ENTIER POSITIF."
            Encod_Bin_Hex_label.value = phrase_hex
    else:
        phrase_dec = "L'encodage, en décimal, est : "
        Encod_Bin_Dec_label.value = phrase_dec 
        phrase_hex = "L'encodage, en hexadécimal, est : "
        Encod_Bin_Hex_label.value = phrase_hex

In [25]:
Nombre_utilisateur_label = widgets.Label(value="Entrez un nombre décimal entier positif :")
Text_multi_ligne1 = "L'encodage, en binaire, est : "
Encod_Dec_Bin_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
Text_multi_ligne2 = "L'encodage, en hexadécimal, est : "
Encod_Dec_Hex_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne2 +' </p>')
Nombre_utilisateur = widgets.Textarea(
    disabled=False
)
Nombre_utilisateur.observe(Dec_Bin_Hex, names='value')
Conv_Dec_Bin = VBox([Nombre_utilisateur_label,Nombre_utilisateur , Encod_Dec_Bin_label, Encod_Dec_Hex_label  ])
Conv_Dec_Bin.layout.max_width = '33%'
Conv_Dec_Bin.layout.min_height = '150px'
Conv_Dec_Bin.layout.border = 'thick double #0066FF'

In [26]:
Nombre_utilisateur_label2 = widgets.Label(value="Entrez un nombre binaire (entier positif) :")
Text_multi_ligne1 = "L'encodage, en décimal, est : "
Encod_Bin_Dec_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne1 +' </p>')
Text_multi_ligne2 = "L'encodage, en hexadécimal, est : "
Encod_Bin_Hex_label = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Text_multi_ligne2 +' </p>')
Nombre_utilisateur2 = widgets.Textarea(
    disabled=False
)
Nombre_utilisateur2.observe(Bin_Dec_Hex, names='value')
Conv_Dec_Bin2 = VBox([Nombre_utilisateur_label2,Nombre_utilisateur2 , Encod_Bin_Dec_label, Encod_Bin_Hex_label  ])
Conv_Dec_Bin2.layout.max_width = '33%'
Conv_Dec_Bin2.layout.min_height = '150px'
Conv_Dec_Bin2.layout.border = 'thick double #33CC33'

In [27]:
Menu = HBox([Conv_Dec_Bin,Conv_Dec_Bin2])
display(Menu)